# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    # from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Creation of a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# KEYSPACE for the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Task 1: Getting the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

### For this query, session_id and item_in_session are the composite keys because they work as filters in the select statement that results in more optimal performance. Also, they are used to make the rows unique.

In [8]:
# Create statement for first query
query = "CREATE TABLE IF NOT EXISTS song_playlist_item_in_session "
query = query + "(session_id int,item_in_session int,artist_name text,song_name text,\
                    song_length float, PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)           

In [9]:
# CSV file provided
file = 'event_datafile_new.csv'

# Inserting data from CSV file to the song_playlist_item_in_session
with open(file, encoding = 'utf8') as f:
    
    # Read CSV file
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # INSERT statement for writing into the song_playlist_item_in_session table
        query = "INSERT INTO song_playlist_item_in_session (session_id , item_in_session , \
                artist_name , song_name , song_length )"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        # Assigning which column element is related for each column in the INSERT statement \
        # using the appropriate data type conversions
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


#### SELECT statement to verify that the data have been inserted into each table

In [10]:
# The SELECT statement to validate the data model for the first query
query = "SELECT artist_name, song_name, song_length from song_playlist_item_in_session  \
           WHERE session_id= 338 and item_in_session = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row[0], row[1], row[2])

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Task 2: Getting the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### For this query user_id, session_id and item_in_session are the composite keys because they make the rows unique. For one thing, user_id and session_id are used as partition keys because they work for filtering in the select statement that results in more optimal performance. On the other hand, item_in_session is used as a clustering column because it is used for ordering columns.

In [11]:
# Create statement for query 2
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text \
                ,song_name text, first_name text, last_name text, \
                PRIMARY KEY ((user_id, session_id), item_in_session)) ;"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                    

In [12]:
# CSV file provided
file = 'event_datafile_new.csv'

# Inserting data from CSV file to the song_playlist_session
with open(file, encoding = 'utf8') as f:
    
    # Read CSV file
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # INSERT statement for writing into the song_playlist_session table
        query = "INSERT INTO song_playlist_session (user_id, session_id, item_in_session , \
                artist_name , song_name , first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Assigning which column element is related for each column in the INSERT statement \
        # using the appropriate data type conversions
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# The SELECT statement to validate the data model for the second query
query = "SELECT artist_name, song_name, first_name, last_name from \
        song_playlist_session WHERE user_id= 10 and session_id = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row[0], row[1], row[2],row[3])

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Task 3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### For this query song_name and user_id are considered as the composite key for uniquely identifying each row. Also, song_name is used as partition key for filtering in the select statement which results in more optimal performance.

In [14]:
# Create statement for query 3
query = "CREATE TABLE IF NOT EXISTS user_playlist_song "
query = query + "(song_name text, first_name text, last_name text, user_id int, \
                PRIMARY KEY ((song_name), user_id) ) ;"

try:
    session.execute(query)
except Exception as e:
    print(e)     
                    

In [15]:
# CSV file provided
file = 'event_datafile_new.csv'

# Inserting data from CSV file to the user_playlist_song
with open(file, encoding = 'utf8') as f:
    
    # Read CSV file
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # INSERT statement for writing into the user_playlist_song table
        query = "INSERT INTO user_playlist_song (song_name , first_name , \
                last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        # Assigning which column element is related for each column in the INSERT statement \
        # using the appropriate data type conversions
        session.execute(query, (line[9], line[1],line[4], int(line[10])))

In [16]:
# The SELECT statement to validate the data model for the third query
query = "SELECT first_name, last_name from user_playlist_song \
        WHERE song_name = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row[0], row[1])

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Dropping the tables before closing out the sessions
query = "DROP TABLE IF EXISTS song_playlist_item_in_session "
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "DROP TABLE IF EXISTS song_playlist_session "
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "DROP TABLE IF EXISTS user_playlist_song "
try:
    session.execute(query)
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [18]:
## Closing the session and cluster connection
session.shutdown()
cluster.shutdown()

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> **_Your question_**